## Website data extraction

In [ ]:
def getMars():
# Dependencies

    from bs4 import BeautifulSoup as bs
    import requests
    import pandas as pd
    from selenium.webdriver import Chrome
    from selenium.webdriver.support.ui import Select
    from selenium import webdriver

    #telling the Chrome driver where is the executable
    driver = Chrome(executable_path='C:/Webdriver/bin/chromedriver')


    #defining a string variable with the website url
    url = "https://mars.nasa.gov/news/"

    #Making a connection to the Nasa News website
    r = requests.get(url)

    #creating a soup object with the html data
    soup = bs(r.text, 'html')

    #Finding all the div tags in the soup
    texto=soup.find_all('div')

    # Opening the website
    driver.get(url)


    # getting the button by class name
    button = driver.find_element_by_class_name("menu_icon")


    # clicking on the button
    button.click()

    #Defining a button for a reactive element which matches the request
    button = driver.find_element_by_id("li_4")


    #click the button
    button.click()

    #saving the current url to a variable for automation
    url2=driver.current_url
    url2

    #Connecting with the News and Events website
    r2=requests.get(url2)

    #creating a soup object with the html text from the Mars Nasa website news and events
    soup2 = bs(r2.text, 'lxml')

    #Looking for all the news and then selecting only the latest news 
    texto2=soup2.find_all('h3', class_="title")
    textoTitle=texto2[0].text.strip()

    #defining the button content (reactive link) for the first news
    button=driver.find_element_by_class_name('list_image')

    #clicking button 
    button.click()

    #Checking the url of the redirected website. Saving to a variable
    url3=driver.current_url
    url3

    #connect to the website that contains the main news
    r3=requests.get(url3)

    #create a soup object for this website
    soup3=bs(r3.text,'lxml')

    #This is the first paragraph of the main text of these news
    texto3=soup3.find('p')
    texto3

    #define a variable with the string of the Space facts website
    url4='https://space-facts.com/mars/'

    #Using requests to connect to Space Facts website
    r4=requests.get(url4)

    #creating a soup element for this website (Mars Facts)
    soup4=bs(r4.text,'html')

    #extracting html from website
    texto4=soup4.find('table')
    #texto4

    #reading the html table from the website Mars Facts
    table=pd.read_html(url4)
    table[0]

    #cresting a dataframe with the information extracted from Mars Facts
    df=pd.DataFrame(table[0])
    df.rename(columns = {0:'Property', 1:'Value'}, inplace = True)
    df

    #Converting the Mars facts to html code
    t_html=pd.DataFrame.to_html(df)
    #t_html


    #website for the hemisphere images of Mars
    url5='https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'



    #executing the driver to automatically go to the website
    driver.get(url5)

    #Create a beautiful soup element with the the html 
    main=bs(r5.text,'lxml')
    titles=main.find_all('div', class_='description')
    #titles



    #collecting the titles of the various images
    In [1]: titlist=[]

    try:
        for tit in titles:
            tititem=tit.find('h3').text
            titlist.append(tititem)

    except:
        print("Something went wrong")
        print(titlist)



    #including a wait for these websites because the response is not fast
    driver.implicitly_wait(10)

    #getting the references from the html script
    elems = driver.find_elements_by_css_selector(".description [href]")
    links = [elem.get_attribute('href') for elem in elems]
    links

    #defining a function to call several times the website and click on each image
    def but1(tag):
       ...:     urln=links[tag]
       ...:     driver.get(urln)
       ...:     button=driver.find_element_by_link_text("Sample")
       ...:     button.click()
       ...:     urlimg=driver.current_url
       ...:     return urlimg

    #Extracting the links to the images from the website (these are not the downloadable ones)
    urlimg=[]
    for u in range(4):
        addurl=but1(u)
        urlimg.append(addurl)

    print(urlimg)



    #assigning a variable to the Mars images website
    url6='https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'

    #testing the response of the website
    r6=requests.get(url6, timeout=10)
    r6

    #inspecting the dataframe for images
    pd.DataFrame({titlist[i]: links[i] for i in range(len(links))}, index=[0])

    hemispheres=[]

    # saving a list of dictionaries with Mars images information
    for i in range(4):
        dic={titlist[i]:links[i]}
        hemispheres.append(dic)

    #print(hemispheres)
    return hemispheres, textoTitle, texto3, t_html


## Mongo and Flask

In [ ]:
#Dependencies
#Flask
from flask import Flask, render_template, redirect
#connection to MongoDb
from pymongo import MongoClient
# import jupyter notebook as a function
import scrape

In [ ]:
#this creates Flask app
Mars_app = Flask(__name__)

In [ ]:
#prepare mongo to upload the images dictionary 
    mongo_url = "mongodb://localhost:27017"
    #pass Mongo URL to MongoClient constructor. Defines the connection
    client = MongoClient(mongo_url)
    #defines a unique handle on the Mongo database for this code
    db = client.MarsPhotos_db
    #handle to the connection to insert the data. Define the variable
    #Mars_collection that will contain the data
    Mars_collection=db.HemImages

In [ ]:
Mars_info=getMars(hemispheres, textoTitle, texto3, t_html)

In [ ]:
def insert_Mars_images():
    Mars_collection.insert_many(hemispheres)
    return 

In [ ]:
def get_my_Mars():
    Mars_collection = get_collection()
    Images = Mars_collection.find()
    return render_template("index.html", HemImages=HemImages)

In [ ]:
# hemispheres=[{"name":"key1Hemisphere", "link":""},{"name":"key2Hemisphere", "link":""},
# {"name":"key3Hemisphere", "link":""},{"name":"key4Hemisphere", "link":""}]

In [ ]:
# def insert_facts_Mongo(): 
#     facts_lst = [{"name":"key1Hemisphere", "link":""},{"name":"key2Hemisphere", "link":""},
#     {"name":"key3Hemisphere", "link":""},{"name":"key4Hemisphere", "link":""}]
#     Mars_collection.insert_many(facts)
#     return

In [ ]:
#This is the route that will import dictionary
@Mars_app.route("/scrape")
def scrape():
    message = "Welcome to my rough Mars API"
    Mars_collection=get_collection()
    return render_template('index.html', text=message, dict=hemispheres)

In [ ]:
@Mars_app.route("/")
def query():
    WhatsNew = db.MarsNews_Mongo.find_one()
    return render_template("index.html", WhatsNew=WhatsNew)


In [ ]:
@Mars_app.route("/bio")
def bio():
    return render_template("bio.html")

In [ ]:
@Mars_app.route("/Sources")
def index():
    Sources_list=["https://mars.nasa.gov/news/",
                  "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars",
                  "https://space-facts.com/mars/"]
    return render_template("sources.html",  Sources=Sources_list)

if __name__ == "__main__":
    insert_Mars_images.run(debug=True)
    Mars_app.run(debug=True)